In [ ]:
# Import pandas library
import pandas as pd

In [ ]:
# Mount colab drive
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
# Load the review data for digital camera
data = pd.read_csv('/content/drive/My Drive/FourthBrain/Project/Amazon_Review_Data/product_data_new.csv')

In [ ]:
data

,Product_Id,Product_Review,Rating,Review_Summary,Helpful_Yes,Helpful_No,Helpfulness_Proportion
0,B00000JBIA,It is absolutely useless. The pictures are gr...,1.0,This camera is HORRIBLE!,24,0,1.000000
1,B00000JBIA,I bought two of these for my kids and they are...,1.0,Just awful; avoid at all costs,5,0,1.000000
2,B00000JBIA,"I bought the Smile to have a handy, inexpensiv...",1.0,Not the best choice,47,0,1.000000
3,B00000JBIA,". STOP!!! . 1/ This camera is NOT 640x480, is ...",1.0,BE CAREFUL!!!,16,1,0.941176
4,B00000JBIA,"I'm on a small budget this summer, so I bought...",1.0,"Worse than I thought, but at that price...",28,14,0.666667
...,...,...,...,...,...,...,...
27378,B00K7O2DJU,PROs and CONsPROs:-Excellent image quality. Th...,5.0,3rd time is (mostly) a charm.,11,0,1.000000
27379,B00K7O2DJU,This is the camera I have been waiting a loooo...,5.0,The perfect little camera when I want high qua...,19,0,1.000000
27380,B00K7O2DJU,"Before I placed my order for this camera, I tr...",3.0,Cannot take still photos while shooting video,13,21,0.382353
27381,B00K7O2DJU,I bought the RX100 in its original configurati...,5.0,A Very Nice Upgrade from Sony - Update 6/22/14,75,1,0.986842


In [ ]:
reviews = data[['Product_Review']]
# Convert to lower cases 
reviews['Product_Review'] = reviews['Product_Review'].str.lower()
# Drop NA's
reviews = reviews.dropna(how='any',axis=0)

# Create review IDs
reviews.reset_index(inplace=True)
reviews = reviews.rename(columns = {'index':'Review_ID'})
reviews.head(5)
# Reset indexes
#reviews = reviews.reset_index()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Review_ID,Product_Review
0,0,it is absolutely useless. the pictures are gr...
1,1,i bought two of these for my kids and they are...
2,2,"i bought the smile to have a handy, inexpensiv..."
3,3,". stop!!! . 1/ this camera is not 640x480, is ..."
4,4,"i'm on a small budget this summer, so i bought..."


In [ ]:
print(reviews.shape)
reviews.iloc[0,1]

(27382, 2)


"it is absolutely useless.  the pictures are grainy, blurry, and have color aberrations galore.  you can't even see facial features on people.  it is not a true 640x480 camera; it uses software to obtain the 640x480 advertised resolution.  downloads are horribly slow, and the memory is not removeable, so you can't use a better download method.  it will drain the batteries with only one loading and unloading of the 2mb memory.  this thing is not even useful as a toy for your children.  fortunately i bought from a reputable online dealer who took the thing back, and even refunded all shipping charges.  do yourself a favor and get an olympus d-360l or d-460z.  ...a lot more money, but about the least you can pay for a digital camera that takes decent pictures.do not buy this camera!"

In [ ]:
# Import NLP libraries 
import nltk
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, RegexpTokenizer, sent_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
print(reviews.iloc[0,1])

it is absolutely useless.  the pictures are grainy, blurry, and have color aberrations galore.  you can't even see facial features on people.  it is not a true 640x480 camera; it uses software to obtain the 640x480 advertised resolution.  downloads are horribly slow, and the memory is not removeable, so you can't use a better download method.  it will drain the batteries with only one loading and unloading of the 2mb memory.  this thing is not even useful as a toy for your children.  fortunately i bought from a reputable online dealer who took the thing back, and even refunded all shipping charges.  do yourself a favor and get an olympus d-360l or d-460z.  ...a lot more money, but about the least you can pay for a digital camera that takes decent pictures.do not buy this camera!


In [ ]:
##### 1. Pre-processing 1: Sentence split 
out_strings = []
for i in range(len(reviews['Product_Review'])): 
  sentences = sent_tokenize(reviews.iloc[i,1])
  out_strings.append(sentences)

print(out_strings[0:5])


[['it is absolutely useless.', 'the pictures are grainy, blurry, and have color aberrations galore.', "you can't even see facial features on people.", 'it is not a true 640x480 camera; it uses software to obtain the 640x480 advertised resolution.', "downloads are horribly slow, and the memory is not removeable, so you can't use a better download method.", 'it will drain the batteries with only one loading and unloading of the 2mb memory.', 'this thing is not even useful as a toy for your children.', 'fortunately i bought from a reputable online dealer who took the thing back, and even refunded all shipping charges.', 'do yourself a favor and get an olympus d-360l or d-460z.', '...a lot more money, but about the least you can pay for a digital camera that takes decent pictures.do not buy this camera!'], ['i bought two of these for my kids and they are awful.', 'the picture quality is so low that only about 10% of the photos are worth keeping.', 'and being for my young children, my stand

In [ ]:
print(len(out_strings))
print(len(reviews))
#print(out_strings[0:10])

27382
27382


In [ ]:
col_names = ['Review_ID', 'Sentence_ID', 'Sentence']
df_sentence_split = pd.DataFrame(columns=col_names)
print(df_sentence_split)

Empty DataFrame
Columns: [Review_ID, Sentence_ID, Sentence]
Index: []


In [ ]:
len(out_strings[0])
print(out_strings[0][0])
print(out_strings[0][1])

it is absolutely useless.
the pictures are grainy, blurry, and have color aberrations galore.


In [ ]:
# Convert list of list to rows of setences with review ID and sentence IDs
col_names = ['Review_ID', 'Sentence_ID', 'Sentence']
df_sentence_split = pd.DataFrame(columns=col_names)

count = 1

for i in range(len(out_strings)): 
  m = len(out_strings[i])
  for j in range(m): 
    df_sentence_split.at[count, 'Review_ID'] = i
    df_sentence_split.at[count, 'Sentence_ID'] = count
    df_sentence_split.at[count, 'Sentence'] = out_strings[i][j]
    count = count + 1

# Check 
df_sentence_split.head(10)


In [ ]:
df_sentence_split.head(300)

In [ ]:
# Save interim sentence split results to csv file 
df_sentence_split.to_csv('review_digital_camera_sentence_split.csv')

In [ ]:
# Test of tokenization and POS tagging with 1 sentence 
tokenizer = RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize(df_sentence_split.at[0, 'Sentence'])
tagged = pos_tag(tokens)
print(tagged)

[('it', 'PRP'), ('is', 'VBZ'), ('absolutely', 'RB'), ('useless', 'JJ')]


In [ ]:
##### 2. Pre-processing 2: Tokenize each setence into words and do part-of-speech (POS) tagging
tokenizer = RegexpTokenizer(r'\w+')
df_sentence_split['Tagged'] = ""

for index in df_sentence_split.index: 
  tokens = tokenizer.tokenize(df_sentence_split.at[index, 'Sentence'])
  tagged = pos_tag(tokens)
  df_sentence_split.at[index, 'Tagged'] = tagged 

# Check the results
df_sentence_split.head(5)

,Review_ID,Sentence_ID,Sentence,tokens,Tagged
0,0,0,it is absolutely useless.,NaN,"[(it, PRP), (is, VBZ), (absolutely, RB), (usel..."
1,1,1,i bought two of these for my kids and they are...,NaN,"[(i, JJ), (bought, VBD), (two, CD), (of, IN), ..."
2,2,2,"i bought the smile to have a handy, inexpensiv...",NaN,"[(i, NN), (bought, VBD), (the, DT), (smile, NN..."
3,3,3,.,NaN,[]
4,4,4,"i'm on a small budget this summer, so i bought...",NaN,"[(i, JJ), (m, NN), (on, IN), (a, DT), (small, ..."


In [ ]:
pd.set_option('display.max_columns', 500)
df_sentence_split.head(20)

,Review_ID,Sentence_ID,Sentence,tokens,Tagged
0,0,0,it is absolutely useless.,NaN,"[(it, PRP), (is, VBZ), (absolutely, RB), (usel..."
1,1,1,i bought two of these for my kids and they are...,NaN,"[(i, JJ), (bought, VBD), (two, CD), (of, IN), ..."
2,2,2,"i bought the smile to have a handy, inexpensiv...",NaN,"[(i, NN), (bought, VBD), (the, DT), (smile, NN..."
3,3,3,.,NaN,[]
4,4,4,"i'm on a small budget this summer, so i bought...",NaN,"[(i, JJ), (m, NN), (on, IN), (a, DT), (small, ..."
5,5,5,"bought the camera mid-june, and by late june w...",NaN,"[(bought, VBD), (the, DT), (camera, NN), (mid,..."
6,6,6,this was my first and only digital camera that...,NaN,"[(this, DT), (was, VBD), (my, PRP$), (first, J..."
7,7,7,i really like mine.,NaN,"[(i, RB), (really, RB), (like, IN), (mine, NN)]"
8,8,8,i bought the kodak dc240 partially on a whim.,NaN,"[(i, NN), (bought, VBD), (the, DT), (kodak, NN..."
9,9,9,this is the best camera i have seen for the pr...,NaN,"[(this, DT), (is, VBZ), (the, DT), (best, JJS)..."


In [ ]:
# Install transformers (HuggingFace)
!pip install transformers

In [ ]:
###### 3. Sentiment Score Model (i.e. Model 1): Import pipeline for sentiment analysis (at sentence level)
from transformers import pipeline
sentiment_analysis = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
# Generate sentiment for one sentence as a test 
example = df_sentence_split['Sentence'][0]
result = sentiment_analysis(example)[0]
print(result['label'])

NEGATIVE


In [ ]:
df_sentence_split.shape

(27434, 5)

In [ ]:
# Apply sentiment analysis for each sentence 
col_name2 = ['Review_ID', 'Sentence_ID', 'Sentence', 'Tagged', 'label', 'score']
sentiment_score = pd.DataFrame(columns=col_name2)
for index in df_sentence_split.index: 
  result = sentiment_analysis(df_sentence_split.at[index, 'Sentence'])[0]
  sentiment_score.at[index, 'Review_ID'] = df_sentence_split.at[index, 'Review_ID']
  sentiment_score.at[index, 'Sentence_ID'] = df_sentence_split.at[index, 'Sentence_ID']
  sentiment_score.at[index, 'Sentence'] = df_sentence_split.at[index, 'Sentence']
  sentiment_score.at[index, 'Tagged'] = df_sentence_split.at[index, 'Tagged']
  sentiment_score.at[index, 'label'] = result['label']
  sentiment_score.at[index, 'score'] = result['score']

Token indices sequence length is longer than the specified maximum sequence length for this model (772 > 512). Running this sequence through the model will result in indexing errors


IndexError: ignored

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')
embeddings = model.encode(sentences)

In [ ]:
tagged_sentence = df_sentence_split.at[0, 'Tagged']
tagged_sentence

AttributeError: ignored

In [ ]:
tagged_sentence = df_sentence_split.at[0, 'Tagged']
for k in range(len(tagged_sentence)): 
   log = (tagged_sentence [k][1][0] == 'N')
    if log == True:
        a.append(tagged [i][0])


In [ ]:
### Pre-processing #3: Only maintain Nouns 
for index in df_sentence_split.index: 
  df_sentence_split.at[index, 'Tagged']

In [ ]:
!pip install ktrain==0.28.2

     |████████████████████████████████| 25.3 MB 1.6 MB/s 
     |████████████████████████████████| 6.8 MB 39.5 MB/s 
     |████████████████████████████████| 981 kB 58.8 MB/s 
     |████████████████████████████████| 263 kB 54.6 MB/s 
     |████████████████████████████████| 2.8 MB 16.5 MB/s 
     |████████████████████████████████| 1.2 MB 61.4 MB/s 
     |████████████████████████████████| 468 kB 54.2 MB/s 
     |████████████████████████████████| 895 kB 55.1 MB/s 
     |████████████████████████████████| 3.3 MB 26.7 MB/s 
     |████████████████████████████████| 59 kB 6.5 MB/s 
     |████████████████████████████████| 596 kB 52.1 MB/s 
  Created wheel for ktrain: filename=ktrain-0.28.2-py3-none-any.whl size=25290606 sha256=63a21bac3f96506b7b2eb443ea1f2fc627204ea6f1f65bc5179ffe8fc00a2302
  Stored in directory: /root/.cache/pip/wheels/df/2b/04/7c821b51e637ec480060989b5030d0c4cce16efe0d67bff94b
  Created wheel for seqeval: filename=seqeval-0.0.19-py3-none-any.whl size=9929 sha256=8832dd191e23bc41

In [ ]:
import ktrain
### 4. Topic modeling 
# Estimate LDA Topic model
tm = ktrain.text.get_topic_model(df_sentence_split['Tagged_Nouns'], n_features=10000)


ImportError: ignored

In [ ]:
# Print topics
tm.print_topics()

In [ ]:
  tokenizer = RegexpTokenizer(r'\w+')
  
  for sentence in out_strings:
    tokens = tokenizer.tokenize(sentence)
    filtered_words = [w for w in tokens if not w.lower() in stop_words]
    ready_for_pos.append((filtered_words))

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
def preprocess(paragraph):
  ready_for_pos = []
  sentence_tokenize = sent_tokenize(paragraph)
  tokenizer = RegexpTokenizer(r'\w+')
  for sentence in sentence_tokenize:
    tokens = tokenizer.tokenize(sentence)
    filtered_words = [w for w in tokens if not w.lower() in stop_words]
    ready_for_pos.append((filtered_words))
  return ready_for_pos 

In [ ]:
combined_sentences = []
for sentences in strings:
  for sentence in sentences:
    combined_sentences.append(" ".join(sentence))

In [ ]:
strings[0][0]

['absolutely', 'useless']

In [ ]:
combined_sentences

['absolutely useless',
 'pictures grainy blurry color aberrations galore',
 'even see facial features people',
 'true 640x480 camera uses software obtain 640x480 advertised resolution',
 'downloads horribly slow memory removeable use better download method',
 'drain batteries one loading unloading 2mb memory',
 'thing even useful toy children',
 'fortunately bought reputable online dealer took thing back even refunded shipping charges',
 'favor get olympus 360l 460z',
 'lot money least pay digital camera takes decent pictures buy camera',
 'bought two kids awful',
 'picture quality low 10 photos worth keeping',
 'young children standards low',
 'bought almost toy still severely disappointed',
 'buy',
 'bought smile handy inexpensive way addingphtos web pages',
 'want spend 400 camera wouldbe obsolete months',
 'figured 100 go wrong',
 'mistaken smile many compromises',
 'lens tiny fixed focus item requires bright light form image',
 'battery life abysmal seems chew pair aas shooting do

In [ ]:
!pip install -U sentence-transformers

     |████████████████████████████████| 78 kB 3.9 MB/s 
     |████████████████████████████████| 3.1 MB 55.9 MB/s 
     |████████████████████████████████| 3.3 MB 59.9 MB/s 
     |████████████████████████████████| 1.2 MB 77.2 MB/s 
     |████████████████████████████████| 59 kB 4.5 MB/s 
     |████████████████████████████████| 596 kB 67.2 MB/s 
     |████████████████████████████████| 895 kB 73.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=950bd5c8232bc00e6ace951f491f3761d4235757b28e378c413ae71bb8df109c
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
strings

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')
embeddings = model.encode(sentences)

In [ ]:
print(embeddings)

[[ 0.60695595  0.1097946  -0.20631394 ... -0.57532674 -0.37692133
   0.25492123]
 [-0.38259122  0.11124049  0.2833771  ... -0.2987023   0.24223335
   0.142224  ]
 [-0.16428566  0.9030792   0.5490557  ... -0.11313283 -0.16936132
   0.00372389]
 ...
 [-0.3107364   0.44211704  0.05907588 ... -0.26294953  0.13869724
   1.0502722 ]
 [ 0.21096036  0.8972505  -0.05371262 ... -0.41855735  0.14375055
   0.3303167 ]
 [ 0.08679213  0.04602114  0.0143583  ... -0.30536556  1.2825226
  -0.15749386]]


In [ ]:
embeddings.shape

(11, 384)